In [3]:
! pip install torch

  Using cached torch-2.8.0-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.8.0-cp313-none-macosx_11_0_arm64.whl (73.6 MB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
Using cached filelock-3.19.1-py3-none-any.whl (15 kB)
Using cached fsspec-2025.9.0-py3-none-any.whl (199 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Us

In [ ]:
import torch
import torch.nn as nn
import math

# Defining Embedding class

class Inputembedding(nn.Module):
    # defining constructor

    def __init__(self,d_model:int,vocab_size:int):
        super().__init__()
        
        self.d_model= d_model # dimension of the word vector
        self.vocab_size= vocab_size # total number of word in vocubalory
        self.embedding = nn.Embedding(vocab_size, d_model)
    
    def forward(self,x):
        return self.embedding(x) *  math.sqrt(self.d_model)

 # positional encoding   
class positionalEncoding(nn.Module):
    def __init__(self,d_model:int,seq_len:int,dropout:float):
        super().__init__()

        self.d_model= d_model
        self.seq_len= seq_len
        self.dropout= nn.Dropout(dropout)
        pe= torch.zeros(seq_len,d_model) # create a matrix of shape. (seq_len,d_model)
        position= torch.arange(0,seq_len,dtype=torch.float).unsqueeze(1) # position shape will be (seq_len,1) it will show position of each word

        div_term= torch.exp(torch.arange(0,d_model,2).float()* (-math.log(10000)/d_model))
        # apply the sin to the even position word
        pe[:,0::2]= torch.sin(position*div_term)
        pe[:,1::2]= torch.cos(position*div_term)
        # Batch dimension = number of sentences (or sequences) processed in parallel.
        # Positional encoding is shared across all sentences in the batch.
        pe= pe.unsqueeze(0) # so it will be dimensioin (1,seq_len,d_model)
        self.register_buffer('pe',pe) # add into register buffer 

    def forward(self,x):
        x= x + (self.pe[:,:x.shape[1],:]).requieres_grad(False)
        return self.dropout(x)


# layer Normalization.....

class LayerNormalization(nn.Module):
    def __init__(self,eps:float=10**-6)->None:
        super().__init__()
        self.eps= eps
        self.alpha= nn.parameter(torch.ones(1))
        self.bias= nn.parameter(torch.zeros(0))

    def forward(self,x):
        mean= x.mean(dim= -1,keepdim=True)
        std= x.std(dim= -1,keepdim=True)
        return self.alpha*(x-mean)/(std+self.eps) + self.bias


# Feed Forword Neural Network class

class FNN(nn.Module):
    def __init__(self,d_model:int,d_ff:int,dropout:float)->None:
        super().__init__()
        self.linear_1= nn.Linear(d_model,d_ff)
        self.dropout= nn.Dropout(dropout)
        self.linear_2= nn.Linear(d_ff,d_model)

    def forward(self,x):
        # (Batch,seq-len,d_model)----> (Batch,seq-len,d_ff)----> (Batch,seq-len,d_model):

        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

        

